In [1]:
%load_ext autoreload
%autoreload 2

In [21]:
import os
import sys
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

sys.path.append(os.path.abspath(os.path.join(os.path.pardir, 'src')))
from jem import JointEmbeddingModel, get_data

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}
import tensorflow as tf
import absl.logging

absl.logging.set_verbosity(absl.logging.ERROR)

# Set memory growth
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

lr = 0.001
epochs = 100
batch_size = 128
num_epochs = 10
input_state_embed = 64
hidden_state_embed = 32
output_state_embed = 16 
exp_embed = 32
output_exp_embed = 16

# Load the data
states, explinations, labels, vocab_size = get_data(batch_size=batch_size)
max_sent_len = explinations.shape[1]

jem = JointEmbeddingModel(vocab_size=vocab_size, max_sent_len=max_sent_len, learning_rate=lr)

jem.fit(states, explinations, labels, batch_size=batch_size, epochs=epochs)

jem.save_model('../models/jem/joint_embedding_model.keras')

Model: "model_19"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_39 (InputLayer)          [(None, 4, 7, 7)]    0           []                               
                                                                                                  
 conv2d_38 (Conv2D)             (None, 4, 7, 4)      256         ['input_39[0][0]']               
                                                                                                  
 conv2d_39 (Conv2D)             (None, 4, 7, 6)      222         ['conv2d_38[0][0]']              
                                                                                                  
 flatten_19 (Flatten)           (None, 168)          0           ['conv2d_39[0][0]']              
                                                                                           

2024-02-15 13:37:16.094935: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-02-15 13:37:16.343210: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_10.
2024-02-15 13:37:16.827197: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-02-15 13:37:17.726485: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 6s 6s/step - loss: 2.3496 - combined_output_loss: 2.3496 - final_state_accuracy: 0.0000e+00 - final_explanation_accuracy: 0.0000e+00 - combined_output_accuracy: 0.0000e+00
Epoch 2/100
1/1 [==============================] - 0s 474ms/step - loss: 1.6211 - combined_output_loss: 1.6211 - final_state_accuracy: 0.0000e+00 - final_explanation_accuracy: 0.0000e+00 - combined_output_accuracy: 0.0000e+00
Epoch 3/100
1/1 [==============================] - 0s 435ms/step - loss: 1.3128 - combined_output_loss: 1.3128 - final_state_accuracy: 0.0000e+00 - final_explanation_accuracy: 0.0000e+00 - combined_output_accuracy: 0.0000e+00
Epoch 4/100
1/1 [==============================] - 0s 488ms/step - loss: 1.1743 - combined_output_loss: 1.1743 - final_state_accuracy: 0.0000e+00 - final_explanation_accuracy: 0.0000e+00 - combined_output_accuracy: 0.0000e+00
Epoch 5/100
1/1 [==============================] - 0s 454ms/step - loss: 1.1051 - combined_output_loss: 1.1051 